In [448]:
import pandas as pd
import pingouin as pg
from scipy.stats import false_discovery_control

import warnings 
warnings.filterwarnings('ignore')

In [449]:
df_main = pd.read_csv('cvd_risk_covars.csv', index_col=0)
#print(df_main.head())
#df_microbiome = pd.read_csv('microbes.csv', index_col=0)

#df_microbiome = pd.read_csv('microbes.csv', index_col=0)
#df_microbiome = pd.read_csv('microbes_filtered_20percent.csv', index_col=0)
#df_microbiome = pd.read_csv('microbes_filtered_10percent.csv', index_col=0)
#df_microbiome = pd.read_csv('microbes_filtered_5percent.csv', index_col=0)
df_microbiome = pd.read_csv('microbes_filtered_2percent.csv', index_col=0)
#df_microbiome = pd.read_csv('microbes_filtered_1percent.csv', index_col=0)
#df_microbiome = pd.read_csv('microbes_filtered_0.5percent.csv', index_col=0)

#df_microbiome = df_microbiome.iloc[:, -50:]
df_microbiome = df_microbiome.apply(pd.to_numeric, errors='coerce')
print(df_microbiome.dtypes.value_counts())
df_microbiome.head()

int64    567
Name: count, dtype: int64


,Peredibacter,Legionella,Lineage_IIa,Brevibacillus,Lineage_IIb,Hydrogenedensaceae,SM2D12,WD2101_soil_group,Eubacterium_oxidoreducens_group,SAR324_clade.Marine_group_B.,...,Subdoligranulum,Escherichia.Shigella,Faecalibacterium,Agathobacter,Blautia,Collinsella,Intestinibacter,Clostridium_sensu_stricto_1,Bifidobacterium,Romboutsia
LS6651,0,0,0,0,0,0,0,0,0,0,...,3769,2920,5327,1519,6516,4055,1072,10954,4498,9112
LS8700,0,0,0,0,0,0,0,0,0,0,...,7529,1845,6716,4483,8035,6316,8560,26816,4974,13406
LS8710,0,0,0,0,0,0,0,0,0,0,...,9829,1764,10090,3152,14819,6179,5324,5997,6572,8469
LS8729,0,0,0,2,0,0,0,0,0,0,...,5318,586,3853,2214,7852,25859,3082,4280,1778,9901
LS8751,0,0,0,0,0,0,0,0,0,0,...,4522,2116,6216,8549,6205,3879,4541,5892,8605,7616


In [450]:
print(df_main['occpcat2'].value_counts())
print(df_main['exc4'].value_counts())

occpcat2
Category 1    91
Category 2    74
Category 3    44
Name: count, dtype: int64
exc4
Not active      116
Very active      46
A bit active     43
Name: count, dtype: int64


In [451]:
df_main['setting'] = df_main['setting'].map({'Rural':0, 'Urban':1})
df_main['sex'] = df_main['sex'].map({'male':0, 'female':1})
df_main['smok'] = df_main['smok'].map({'No':0, 'Yes':1})
df_main['alcoh'] = df_main['alcoh'].map({'0':0, 'Yes':1})
df_main['infection'] = df_main['infection'].map({'Negative':0, 'Positive':1})
df_main['occpcat2'] = df_main['occpcat2'].map({'Category 1':0, 'Category 2':1, 'Category 3':2})
df_main['exc4'] = df_main['exc4'].map({'Not active':0, 'A bit active':1, 'Very active':2})

df_main.head()

,slabno,setting,age,sex,bmi,occpcat2,exc4,smok,alcoh,infection,insulin,BP_Dia,Chol_T,Chol_L,glucose,BP_Sys
LS6651,LS6651,0,31,0,26.55186,1,0.0,0.0,0.0,1,3.72,84.0,4.99,3.24,4.52,121.0
LS8700,LS8700,0,42,0,20.91135,2,0.0,0.0,1.0,1,8.22,72.5,4.19,2.53,5.60,103.0
LS8710,LS8710,0,19,0,22.52859,1,2.0,0.0,0.0,1,4.41,70.5,3.41,2.20,4.00,120.0
LS8729,LS8729,0,60,0,21.35779,1,1.0,0.0,1.0,0,2.93,71.5,4.93,3.12,4.65,119.0
LS8751,LS8751,0,32,0,23.42209,1,0.0,0.0,0.0,1,3.44,60.5,3.46,2.19,4.46,120.0


In [452]:
# Define your CVD outcomes and covariates
cvd_outcomes = ['insulin', 'BP_Dia', 'Chol_T', 'Chol_L', 'glucose', 'BP_Sys']
covariates = ['age', 'sex',	'bmi',	'occpcat2',	'exc4',	'smok',	'alcoh', 'infection']

# Get microbe columns (or specify a subset)
microbe_columns = df_microbiome.columns.tolist()

In [453]:
df = pd.concat([df_main, df_microbiome], axis=1)
#print(df.columns)
print(df.dtypes)
df.tail()
print(df['setting'].value_counts())

slabno                          object
setting                          int64
age                              int64
sex                              int64
bmi                            float64
                                ...   
Collinsella                      int64
Intestinibacter                  int64
Clostridium_sensu_stricto_1      int64
Bifidobacterium                  int64
Romboutsia                       int64
Length: 583, dtype: object
setting
0    127
1     82
Name: count, dtype: int64


In [454]:
# for microbe in microbe_columns:
#     print(df[microbe].value_counts())

In [455]:
covariates = ['age', 'bmi', 'sex', 'infection', 'alcoh', 'smok', 'occpcat2', 'exc4']
microbe_cols = microbe_columns
cvd_vars = cvd_outcomes

results = pd.DataFrame()

cvd_vars = ['BP_Sys']

for cvd_outcome in cvd_vars:
    print(f"Processing {cvd_outcome}...")
    
    for i, microbe in enumerate(microbe_cols):
            
            # Run mediation
            med = pg.mediation_analysis(
                data=df,
                x='setting',  # rural/urban (binary: 0/1)
                m=microbe,      # mediator
                y=cvd_outcome,
                covar=covariates
            )
            med['Microbe'] = microbe
            med['cvd_outcome'] = cvd_outcome

            results = pd.concat([results, med])
            #results.append(med)

Processing BP_Sys...


In [456]:
results

,path,coef,se,pval,CI[2.5%],CI[97.5%],sig,Microbe,cvd_outcome
0,Peredibacter ~ X,-0.148921,0.088287,9.327885e-02,-0.323063,0.025222,No,Peredibacter,BP_Sys
1,Y ~ Peredibacter,-0.870597,1.703316,6.098579e-01,-4.230322,2.489128,No,Peredibacter,BP_Sys
2,Total,2.958050,2.084195,1.574482e-01,-1.152946,7.069047,No,Peredibacter,BP_Sys
3,Direct,2.870534,2.104527,1.741874e-01,-1.280706,7.021773,No,Peredibacter,BP_Sys
4,Indirect,0.087517,0.263880,7.120000e-01,-0.343341,0.645673,No,Peredibacter,BP_Sys
...,...,...,...,...,...,...,...,...,...
0,Romboutsia ~ X,8598.835157,1138.170253,1.691531e-12,6353.837596,10843.832719,Yes,Romboutsia,BP_Sys
1,Y ~ Romboutsia,0.000084,0.000117,4.743258e-01,-0.000147,0.000314,No,Romboutsia,BP_Sys
2,Total,2.958050,2.084195,1.574482e-01,-1.152946,7.069047,No,Romboutsia,BP_Sys
3,Direct,2.907539,2.381514,2.236450e-01,-1.790064,7.605142,No,Romboutsia,BP_Sys


In [457]:
# Apply FDR correction
results['Indirect_pval_fdr'] = false_discovery_control(results['pval'])

In [458]:
results_indirect = results[results['path']== 'Indirect']
results_indirect = results_indirect[results_indirect['sig']== 'Yes']
results_indirect

,path,coef,se,pval,CI[2.5%],CI[97.5%],sig,Microbe,cvd_outcome,Indirect_pval_fdr
4,Indirect,0.566693,0.305246,0.004,0.091946,1.258678,Yes,Roseomonas,BP_Sys,0.112277
4,Indirect,0.253842,0.183595,0.048,0.010967,0.692617,Yes,UCG.008,BP_Sys,0.333784
4,Indirect,0.391468,0.260764,0.012,0.035585,1.015311,Yes,BIyi10,BP_Sys,0.223205
4,Indirect,0.364271,0.221313,0.044,0.046997,0.960259,Yes,Tabrizicola,BP_Sys,0.333784
4,Indirect,-0.129338,0.125096,0.036,-0.376389,0.005359,Yes,Georgenia,BP_Sys,0.333784
4,Indirect,-0.204157,0.367480,0.016,-0.648311,0.064468,Yes,Neisseria,BP_Sys,0.254831
4,Indirect,0.627319,0.358629,0.028,0.047959,1.387277,Yes,Negativibacillus,BP_Sys,0.333784
4,Indirect,-0.103396,0.294415,0.024,-0.278220,0.213514,Yes,Allobaculum,BP_Sys,0.333303
4,Indirect,2.201259,0.925725,0.004,0.380566,4.079395,Yes,Lachnospiraceae_UCG.004,BP_Sys,0.112277
4,Indirect,0.321389,0.169930,0.044,0.040205,0.720537,Yes,Hungatella,BP_Sys,0.333784


In [459]:
results_indirect.shape

(13, 10)

In [460]:
results_indirect.to_csv('B_BP_Sys_mediation.csv', index=False)

In [461]:
#df_main.exc1.value_counts()

In [462]:
#df_main.occp.value_counts()